In [2]:
import pandas as pd
import glob
import os
import re
from datetime import datetime

In [10]:
#convert dates that are Str into date type
#if the date is a String it is formated as "24.12." 
#year gets extracted from filename: 2 possible types XX or XXYY

def convert_date(year, date):
    #format example: "03.05.2007"
    format = "%d.%m.%y"

    try:
        #is date a string and year of type XX?
        if (type(date) == str) and (len(year) == 2):
            date = date + year

        #is date a string and year of type XXYY?
        elif (type(date) == str) and (len(year) > 2): 
            #XXYY (len = 4) implies that it's a wintersemester (starts 01.10, ends 01.04)
            #if month is bigger than 6 choose XX
            if int(date[3:-1]) > 6:
                date = date + year[:-2]
            #else choose YY as year
            else:
                date = date + year[2:]

        #if non of the above is true, date is already date type
        else:
            return date 
        
        #turn the formated date string into date type and return
        return datetime.strptime(date, format)
    
    except: 
        #catched exception means that final date string could not be converted to date type
        #which implies that we want to ignore this later (could be info in date column that are no dates)
        return float('NaN')

In [26]:
#remove unwanted characters from the title
def clean_title(title):
    #list of words we want to delete
    naughty_list = [' \(OmU\)', " \(OV\)", " OmU", " OmeU", " OV"]
    #check titles and remove if something matches
    for scratch in naughty_list:
        title = re.sub(scratch, "", title)
    return title

In [98]:
def extract_wanted(df, year):
    #convert_dates
    df['Datum'] = df.apply(lambda x: convert_date(year, x['Datum']), axis=1)
    
    #only use rows with dates
    df = df.dropna(subset= 'Datum')
    
    #only keep date, name, attendance
    df = df[["Datum", "Titel", "Zuschauer"]]
    
    #New Column Names
    df.columns = ['Date', "Titel", "Attendance"]
    
    #Convert Attendance Type from float to int
    df = df.dropna(subset= "Attendance")
    df['Attendance'] = df['Attendance'].astype(int)
    
    #remove words that are not part of the title
    df['Titel'] = df['Titel'].apply(clean_title)
    return df

In [101]:
def export_to_csv(folder, extension):
    for file in glob.glob(f"{folder}*{extension}"):
        try:
            #normalize file name
            file_name = os.path.basename(file).lower()
            #remove extension and format for export
            file_name = os.path.splitext(file_name)[0]
            file_name = re.sub(" ", "_", file_name)

            #last part of file name is the year
            year = re.split("_", file_name)[1]
            
            #read file into dataframe
            df = pd.read_excel(file)
            
            #cut and format dataframe appropriatly 
            content = extract_wanted(df, year)
            
            #export dataframe as .csv file 
            content.to_csv(f"/home/tisinti/Projekte/Movie_Attendence_Prediction/Uni_Data_Semesters/{file_name}.csv", encoding='utf-8', index = False)
        
        except:
            print(f"{file_name} did not go through.")

In [102]:
folder = "/home/tisinti/Projekte/Movie_Attendence_Prediction/Besucherzahlen/"
extension = [".xls", ".xlsx", ".ods"]

for ext in extension:
    export_to_csv(folder, ext)